In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

import matplotlib.pyplot as plt 
import matplotlib 
matplotlib.rcParams.update({'font.size': 14})
# 'text.usetex': True, 'font.family': 'Helvetica'
%matplotlib inline

from utils.models import MLP
from utils.trainer import train_MLP
from utils.data import create_xor_dataset
from utils.plotting import plot_var

import copy